In [28]:
# Requests for handling HTTP get and other requests
import requests
import time
import pandas as pd
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
# 
from bs4 import BeautifulSoup


In [2]:
url = "https://www.ss.com/lv/real-estate/flats/riga/"
baseurl = "https://ss.com"
postfix = "sell/"

In [24]:
def getRegionList(url, baseurl = "https://ss.com", postfix = "sell/"):
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    allanchors = soup.find_all('a')
    acats = [el for el in allanchors if 'class' in el.attrs\
             and 'a_category' in el.attrs['class']]
    regionlist = [baseurl + el.attrs['href'] + postfix for el in acats]
#     regiontuples = [(el.text, baseurl + el.attrs['href'] + postfix)\
#                     for el in acats]
    # we do not want the last one with all
    return regionlist[:-1]
    

In [25]:
myregions = getRegionList(url)
len(myregions)

52

In [22]:
def getAllLocalUrls(url):
    """Get a list of all urls including paginated pages"""
    results = [url] # default is just the url if no extra pages found
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    # we just need a one element
    prevanchor = soup.find('a', {"rel":"prev"})
    if prevanchor == None:
        return results
    href = prevanchor.attrs.get('href')
    lastPageNum = int(href.split('page')[-1].split('.')[0])
    print(lastPageNum)
    nurls = [f"{url}page{n}.html" for n in range(2,lastPageNum+1)]
    results += nurls
    return results

In [26]:
myresults = getAllLocalUrls(myregions[0])
myresults

26


['https://ss.com/lv/real-estate/flats/riga/centre/sell/',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page2.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page3.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page4.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page5.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page6.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page7.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page8.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page9.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page10.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page11.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page12.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page13.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sell/page14.html',
 'https://ss.com/lv/real-estate/flats/riga/centre/sel

In [15]:
mysplit = href.split(".")
mysplit

['/lv/real-estate/flats/riga/centre/sell/page26', 'html']

In [16]:
splitbypage = mysplit[0].split("page")
splitbypage

['/lv/real-estate/flats/riga/centre/sell/', '26']

In [19]:
int(splitbypage[1]),splitbypage[1],type(splitbypage[1])

(26, '26', str)

In [29]:
def getAllUrls(url="https://www.ss.com/lv/real-estate/flats/riga/"):
    urlist = []
    reglist = getRegionList(url)
    for region in reglist:
        print(f"Working on {region}")
        regurls = getAllLocalUrls(region)
        urlist += regurls
        time.sleep(0.3)
    return urlist

In [30]:
urlist = getAllUrls()
len(urlist)

Working on https://ss.com/lv/real-estate/flats/riga/centre/sell/
26
Working on https://ss.com/lv/real-estate/flats/riga/agenskalns/sell/
5
Working on https://ss.com/lv/real-estate/flats/riga/aplokciems/sell/
Working on https://ss.com/lv/real-estate/flats/riga/beberbeki/sell/
Working on https://ss.com/lv/real-estate/flats/riga/bergi/sell/
Working on https://ss.com/lv/real-estate/flats/riga/bierini/sell/
Working on https://ss.com/lv/real-estate/flats/riga/bolderaya/sell/
Working on https://ss.com/lv/real-estate/flats/riga/breksi/sell/
Working on https://ss.com/lv/real-estate/flats/riga/bukulti/sell/
Working on https://ss.com/lv/real-estate/flats/riga/bulli/sell/
Working on https://ss.com/lv/real-estate/flats/riga/chiekurkalns/sell/
Working on https://ss.com/lv/real-estate/flats/riga/darzciems/sell/
Working on https://ss.com/lv/real-estate/flats/riga/daugavgriva/sell/
Working on https://ss.com/lv/real-estate/flats/riga/dreilini/sell/
Working on https://ss.com/lv/real-estate/flats/riga/dze

122

In [31]:
def getAllRows(url):
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    allrows = soup.find_all('tr')
    adrows = [el for el in allrows\
              if 'id' in el.attrs and 'tr_' in el.attrs['id']]
    return adrows[:-1] # since the last one is not an ad

In [32]:
other = getAllRows("https://ss.com/lv/real-estate/flats/riga/other/sell/")
len(other)

2

In [33]:
def getRow(row):
    # get all table data cells
    tds = row.find_all('td')
    adlist = []
    adlist.append(tds[1].find('a').attrs['href'])
    for td in tds[2:]:
        adlist.append(td.text)
    return tuple(adlist)

In [34]:
getRow(other[0])

('/msg/lv/real-estate/flats/riga/other/blpgn.html',
 'Ogļu 32-1. Īpašnieks pārdod ekskluzīvo mebelētu 3-istabu dzīvokl',
 'Ogļu 32',
 '3',
 '79',
 '1/4',
 'Renov.',
 '3,291 €',
 '260,000  €')

In [48]:
def processAllUrls(urlist):
    results = []
    for url in urlist:
        print(f"Processing {url}")
        rows = getAllRows(url)
        for row in rows:
            results.append(getRow(row))
        time.sleep(0.3)
    return results

In [49]:
myrooms = processAllUrls(urlist)
len(myrooms)

Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page2.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page3.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page4.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page5.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page6.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page7.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page8.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page9.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page10.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page11.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page12.html
Processing https://ss.com/lv/real-estate/flats/riga/centre/sell/page13.html
Processing https://ss.com/lv/re

Processing https://ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/
Processing https://ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page2.html
Processing https://ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page3.html
Processing https://ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page4.html
Processing https://ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page5.html
Processing https://ss.com/lv/real-estate/flats/riga/zolitude/sell/
Processing https://ss.com/lv/real-estate/flats/riga/zolitude/sell/page2.html
Processing https://ss.com/lv/real-estate/flats/riga/zolitude/sell/page3.html
Processing https://ss.com/lv/real-estate/flats/riga/vef/sell/
Processing https://ss.com/lv/real-estate/flats/riga/other/sell/


2740

In [50]:
myrooms[:5]

[('/msg/lv/real-estate/flats/riga/centre/adkci.html',
  'Владелец начинает продажу квартир в здании проектированном знаме',
  'Avotu 4',
  '1',
  '26',
  '2/5',
  'P. kara',
  '1,661 €',
  '43,190  €'),
 ('/msg/lv/real-estate/flats/riga/centre/cecjd.html',
  'Pārdodu mansarda dzīvokli tikko pēc kapitālā remonta:\r\n\r\n+ koka ',
  'Miera 105',
  '3',
  '71',
  '6/6',
  'Renov.',
  '1,823 €',
  '129,402  €'),
 ('/msg/lv/real-estate/flats/riga/centre/bepeij.html',
  'Pārdod 48, 7 kvadrātmetrus plašu divu izolētu istabu dzīvokli, k',
  'Alūksnes 2',
  '2',
  '49',
  '3/5',
  'Staļina',
  '1,388 €',
  '68,000  €'),
 ('/msg/lv/real-estate/flats/riga/centre/bmghm.html',
  'Kompakts 3 istabu dzīvoklis renovētā ēkā. \r\nDzīvoklis ir izvieto',
  'Blaumaņa 12',
  '3',
  '64',
  '4/6',
  'Renov.',
  '2,382 €',
  '152,475  €'),
 ('/msg/lv/real-estate/flats/riga/centre/ebjkm.html',
  'Pārdod 4 istabu dzīvokli Rīgas vēsturiskajā centrā, Dzirnavu iel',
  'Dzirnavu 66A',
  '4',
  '102',
  '2/6',
  'P.

In [42]:
len(myrooms)/9

2741.0

In [46]:
# originally we messed up and used += instead of mylist.append(newitem)
fixmyrooms = [tuple(myrooms[n*9:n*9+9]) for n in range(len(myrooms)//9)]
len(fixmyrooms)

2741

In [47]:
fixmyrooms[:5]

[('/msg/lv/real-estate/flats/riga/centre/adkci.html',
  'Владелец начинает продажу квартир в здании проектированном знаме',
  'Avotu 4',
  '1',
  '26',
  '2/5',
  'P. kara',
  '1,661 €',
  '43,190  €'),
 ('/msg/lv/real-estate/flats/riga/centre/cecjd.html',
  'Pārdodu mansarda dzīvokli tikko pēc kapitālā remonta:\r\n\r\n+ koka ',
  'Miera 105',
  '3',
  '71',
  '6/6',
  'Renov.',
  '1,823 €',
  '129,402  €'),
 ('/msg/lv/real-estate/flats/riga/centre/bepeij.html',
  'Pārdod 48, 7 kvadrātmetrus plašu divu izolētu istabu dzīvokli, k',
  'Alūksnes 2',
  '2',
  '49',
  '3/5',
  'Staļina',
  '1,388 €',
  '68,000  €'),
 ('/msg/lv/real-estate/flats/riga/centre/bmghm.html',
  'Kompakts 3 istabu dzīvoklis renovētā ēkā. \r\nDzīvoklis ir izvieto',
  'Blaumaņa 12',
  '3',
  '64',
  '4/6',
  'Renov.',
  '2,382 €',
  '152,475  €'),
 ('/msg/lv/real-estate/flats/riga/centre/ebjkm.html',
  'Pārdod 4 istabu dzīvokli Rīgas vēsturiskajā centrā, Dzirnavu iel',
  'Dzirnavu 66A',
  '4',
  '102',
  '2/6',
  'P.

In [58]:
base  = "https://www.ss.com"
fixmyrooms = [tuple([base+el[0]] + list(el[1:])) for el in myrooms]
fixmyrooms[:3]

[('https://www.ss.com/msg/lv/real-estate/flats/riga/centre/adkci.html',
  'Владелец начинает продажу квартир в здании проектированном знаме',
  'Avotu 4',
  '1',
  '26',
  '2/5',
  'P. kara',
  '1,661 €',
  '43,190  €'),
 ('https://www.ss.com/msg/lv/real-estate/flats/riga/centre/cecjd.html',
  'Pārdodu mansarda dzīvokli tikko pēc kapitālā remonta:\r\n\r\n+ koka ',
  'Miera 105',
  '3',
  '71',
  '6/6',
  'Renov.',
  '1,823 €',
  '129,402  €'),
 ('https://www.ss.com/msg/lv/real-estate/flats/riga/centre/bepeij.html',
  'Pārdod 48, 7 kvadrātmetrus plašu divu izolētu istabu dzīvokli, k',
  'Alūksnes 2',
  '2',
  '49',
  '3/5',
  'Staļina',
  '1,388 €',
  '68,000  €')]

In [59]:
df = pd.DataFrame(fixmyrooms)
df.head()

,0,1,2,3,4,5,6,7,8
0,https://www.ss.com/msg/lv/real-estate/flats/ri...,Владелец начинает продажу квартир в здании про...,Avotu 4,1,26,2/5,P. kara,"1,661 €","43,190 €"
1,https://www.ss.com/msg/lv/real-estate/flats/ri...,Pārdodu mansarda dzīvokli tikko pēc kapitālā r...,Miera 105,3,71,6/6,Renov.,"1,823 €","129,402 €"
2,https://www.ss.com/msg/lv/real-estate/flats/ri...,"Pārdod 48, 7 kvadrātmetrus plašu divu izolētu ...",Alūksnes 2,2,49,3/5,Staļina,"1,388 €","68,000 €"
3,https://www.ss.com/msg/lv/real-estate/flats/ri...,Kompakts 3 istabu dzīvoklis renovētā ēkā. \r\n...,Blaumaņa 12,3,64,4/6,Renov.,"2,382 €","152,475 €"
4,https://www.ss.com/msg/lv/real-estate/flats/ri...,Pārdod 4 istabu dzīvokli Rīgas vēsturiskajā ce...,Dzirnavu 66A,4,102,2/6,P. kara,"1,765 €","180,000 €"


In [60]:
df.to_csv('apartments.csv')

In [62]:
df.to_excel('apartments.xlsx')

In [74]:
def getColumnNames(url, colNames = ['URL', 'Description']):
    result = []
    result += colNames
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    columnrow = soup.find('tr', {'id':'head_line'})
    tds = columnrow.find_all('td')
    for td in tds[1:]:
        result.append(td.text)
    return result
    

In [75]:
columnNames = getColumnNames("https://www.ss.com/lv/real-estate/flats/riga/other/sell/")

In [76]:
len(columnNames)

9

In [77]:
columnNames

['URL',
 'Description',
 'Iela',
 'Ist.',
 'm2',
 'Stāvs',
 'Sērija',
 'Cena, m2',
 'Cena']

In [78]:
df.columns = columnNames
df.head()

,URL,Description,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,https://www.ss.com/msg/lv/real-estate/flats/ri...,Владелец начинает продажу квартир в здании про...,Avotu 4,1,26,2/5,P. kara,"1,661 €","43,190 €"
1,https://www.ss.com/msg/lv/real-estate/flats/ri...,Pārdodu mansarda dzīvokli tikko pēc kapitālā r...,Miera 105,3,71,6/6,Renov.,"1,823 €","129,402 €"
2,https://www.ss.com/msg/lv/real-estate/flats/ri...,"Pārdod 48, 7 kvadrātmetrus plašu divu izolētu ...",Alūksnes 2,2,49,3/5,Staļina,"1,388 €","68,000 €"
3,https://www.ss.com/msg/lv/real-estate/flats/ri...,Kompakts 3 istabu dzīvoklis renovētā ēkā. \r\n...,Blaumaņa 12,3,64,4/6,Renov.,"2,382 €","152,475 €"
4,https://www.ss.com/msg/lv/real-estate/flats/ri...,Pārdod 4 istabu dzīvokli Rīgas vēsturiskajā ce...,Dzirnavu 66A,4,102,2/6,P. kara,"1,765 €","180,000 €"


In [80]:
df.to_excel('apartments.xlsx')